# Problem

Real estate is changing. Deep learning is leading the way to many new methods of determing how to price a property at a competitive rate. Traditionally, when a seller is ready to put a property on the market, they would contact an agent to start the process. In recent times, this process has changed; we now have iBuyers in the market that can harness the details (They ask sellers fill out a short form online or accept a call to answer questions about your home features, updates and condition, photos to make you the seller a compepetitive cash offer).  
Real estate is complex. Multiple of levels of complexity e.g transactions, additions, updates. Zingat operates as an intermediary between potential buyers and sellers in the real estate market. In this regard, to help both parties they routinely value and monitor real estate market. For this purpose, They regularly build new and better models using new technologies in the field of data science. Not only with automated valuation models that predict house prices, they also employ time series models to understand market trends. 
Zingnat has made available a public dataset on kaggle (https://www.kaggle.com/zingatbi/zingat-real-estate) to build our version of a basic Automated Valuation Model. We will use modern methods to predict the price of a property using core principles of Comparable Market Analysis (our basic assumption is that a competitve offer price is a price at which similar or comparable properties in the same area recently sold).
Based on this assumption we will introduce the idea of merging multiple models to output an offer amount for a property. 
We will test these model restricted set of future data to glean the effectiveness of our generalizations.
